In [1]:
using Plots
using Pickle
using PyCall
using StatsPlots
using Distributions
using BenchmarkTools
using Statistics

In [2]:
cd("Desktop/Procesy")

# Cele:

## Zadanie 1

1. Załaduj plik trajektorii.OK
2. Znajdź rozkład zmiennej $X_i$.OK
3. Znajdź intensywność $\lambda$ jednorodengo procesu Poissona.OK
4. Znajdź parametr wysokości premii $\theta$. OK
5. Wygeneruj 100 trajektorii procesu Ryzyka. OK
6. Porównaj je z dostarczonymi plikami. OK 
7. Oblicz prawdopodobieństwo bankructwa dla skończonego czasu. OK
8. Oblicz prawdopodobieństwo brankructwa dla nieskońcoznego czasu. OK

### Zmienna losowa z rozkładu Poissona

In [3]:
function poisson(λ)
    
    i = 0
    p = ℯ^(-λ)
    F = p
    
    U = rand()
    
    while U>F
        p *= λ/(i+1)
        F += p
        i += 1
    end   
        
    return i
    
end

poisson (generic function with 1 method)

### Próba losowa z rozkładu Poissona

In [4]:
function upoisson(λ, size=1)
    
    if size == 1
        return poisson(λ)
        
    else
        
        Values = zeros(size)
        
        for index in 1:size
            Values[index] = poisson(λ)
        end
    end
    
    return Values
    
end

upoisson (generic function with 2 methods)

### Zmienna losowa z rozkładu wykładniczego

#### Gęstość teoretyczna

In [5]:
function pdfexponential(t, μ)
    return μ*ℯ^(-t*μ)
end

pdfexponential (generic function with 1 method)

#### Dystrybuanta teoretyczna

In [6]:
function cdfexponential(t, μ)
    return 1 - ℯ^(-t*μ)
end

cdfexponential (generic function with 1 method)

#### Generator

In [7]:
function exponential(μ)
    U = rand()
    return -log(U)/μ
end

exponential (generic function with 1 method)

### Próba losowa z rozkładu wykładniczego

In [8]:
function rexponential(μ, size=1)
    
    if size == 0
        return [0]
    
    elseif size == 1
        return [exponential(μ)]
        
    else
        
        Values = zeros(size)
        
        for index in 1:size
            Values[index] = exponential(μ)
        end
        
    end
   
    return Values
    
end

rexponential (generic function with 2 methods)

## Generator jednorodnego procesu Poissona z intensywnością $\lambda$ na odcinku [0,T]

### Wartość oczekiwana

In [9]:
function Epoiss(λ, t)
    return λ*t
end

Epoiss (generic function with 1 method)

### Generator

In [10]:
function poisson_process(λ, T)
    
    I = poisson(λ*T)
    
    if I == 0
        return [0]
    
    else
        
        U = zeros(I+1)
        
        for i in 2:I+1
            U[i] = T*rand()
        end
    end
    
    return sort(U)
end

poisson_process (generic function with 1 method)

### Przykładowa trajektoria

In [11]:
X = poisson_process(1, 100)
Y = 1:length(X)

plot(X, Y, 
    linetype=:steppost, 
    legend=:topleft, 
    label="trajektoria", 
    ylabel="liczba skoków", 
    xlabel="czas t",
    color =:blue)
plot!([X[length(X)], 100], [Y[length(X)], Y[length(X)]], color =:blue, label="")

savefig("trajektoria_poissona.pdf")

### Sprawdzenie poprawności

In [12]:
λ = 1
T = 100

100

In [13]:
X = poisson_process(λ, T)
Y = collect(1:length(X))

plot(X, Y, linetype=:steppost, legend = false, color=:blue, label="wiązka trajektorii")

for i in 1:100
    X = poisson_process(λ, T)
    Y = collect(1:length(X))
    
    plt = plot!(X, Y, linetype=:steppost, color=:blue, label="")

end

plt = plot!(0:T, x -> Epoiss(λ, x), color =:red, linewidth=3, xlabel="czas t", ylabel="liczba skoków", label="średnia procesu")

savefig("poprawność_poissona.pdf")

## Proces ryzyka

 Proces ryzyka to proces stochastyczny opisujący kapitał firmy ubezpieczeniowej. Wyraża się on wzorem

## $$R(t) = u + c(t) - \sum_{i}^{N(t)}X_i,$$

gdzie: 
<li>$u>0$ - kapitał początkowy,
<li>$c(t)$ - premia (przychody ze sprzedaży polis)
<li>$N(t)$ - proces liczący straty
<li>$X_i$ - zmienna losowa <i>i.i.d </i>, $EX_1 = \mu$, $X_i >0$, reprezentatywna wartość wypłacanych odszkodowań.

## Pojedyncza trajektoria dla wybranych parametrów

### parametry

In [14]:
u = 50 #kapitał początkowy
θ = 0.5 #narzut
μ = 1 #wartość oczekiwana zmiennej losowej X
λ = 1 #intensywność procesu Poissona
α = (1+θ)*μ*λ #stała w klasycznym procesie ryzyka
c(t) = α*t #funkcja c(t) w klaysycznym procesie ryzyka
T = 100 #Horyzont czasowy
h = 0.01 #krok w symulacji

0.01

### generator (klasyczny proces ryzyka z $X_i$ $\sim$ $Exp(1/\mu)$)

In [15]:
function pay(Pays, index)
    if index == 1
        return 0
    else
        return sum(Pays[1:(index-1)])
    end
end

pay (generic function with 1 method)

In [16]:
function check(Moments, Pays, t)
    
    j::Int = 1
    k::Int = length(Moments)
    
    while j < k-1
        
        if t >= Moments[j] && t <= Moments[j+1]
            return pay(Pays, j)
        else
            j += 1
    
        end
   
    end

    return sum(Pays)
    
end
            

check (generic function with 1 method)

In [17]:
function risk_process(u::Real; θ::Float64=0.5, μ::Float64=1.0, λ::Float64=1.0, T::Int=100, h::Float64=0.01)
    
    N = poisson_process(λ, T) #momenty skoków
    n = length(N)-1 #ilość skoków
    Xs = rexponential(μ, n) #próba losowa z rozkładu wykładniczego - wypłaty odszkodowań
    
    size::Int = round(T/h)
    R = zeros(size)
    
    t = 0
    I = 1
    J = 2
    
    α = (1+θ)*λ/μ
    c(t, a) = α*t
    
    while t <= T
        R[I] = u + c(t, α) - check(N, Xs, t)
        t += h
        I += 1
        
    end
    
    return R
    
end

risk_process (generic function with 1 method)

### Przykładowa trajektoria

In [18]:
xs = 0:h:T-h
ys = risk_process(100, λ=0.7, μ=0.2)

plot(xs,
    ys,
    xlabel="Czas t",
    ylabel="Kapitał firmy ubezpieczeniowej",
    color =:blue,
    legend=:false)

savefig("trajektoria_ryzyka.pdf")

### Oszacowanie parametru $\lambda$

Pomysł: Jeśli $N(t)$ to jednorododny proces Poissona z parametrem intensywności $\lambda$, to czasy oczekiwania na kolejny skok są z rozkładu wykładniczego z tym samym parametrem (przedstawić dowód). W związku z tym należy znaleźć momenty tych skoków i różnice między nimi. Otrzymamy prostą próbę losową z rozkładu wykładniczego z parametrem $\lambda$.

Następnie, za pomocą metody momentów oraz największej wiarogodności wyestymujemy porządany parametr.

### Wektor momentów skoków

Trajektoria to pewien wektor przedstawiający kapitał firmy ubezpieczeniowej w pewnym czasie $\tau$. Oczywiście do momentu wypłaty odszkodowania musi być ona rosnąca. Tym faktem posłużymy się znajdując momenty skoków.

Napiszemy algorytm, który zapisze czasy $\tau^*$, w których nastąpił jakikolwiek spadek w kapitale. Zakładamy przy tym, że trajektoria jest lewostronnie ciągła.

Algorytm

1. Ustal $\tau=0$, $h=0.01$, $I = 1$, trajektorię ryzyka $R$ oraz pusty wektor $V.$
2. Zwiększ czas $\tau$ o krok $h$, zwiększ $I$ o jeden.
3. Sprawdź, czy firma odnotowała spadek ($R[I] < R[I-1]$). Jeśli tak, wstaw $\tau$ do wektora $V$. W przeciwnym przypadku kontynuuj.
4. Powtarzaj 2-3, dopóki $I < \#R$.
5. Zwróć $V.$

In [19]:
function get_moments(trajectory; h=0.01, T=100)
    
    t = 0
    I = 1
    V = []
    k = length(trajectory)
    
    while I < k
        
        t += h
        I += 1
        
        if trajectory[I] < trajectory[I-1]
            push!(V, t)
        else
            continue
        end

    end
    
    return V

end

get_moments (generic function with 1 method)

In [20]:
function get_times_from_trajectories(matrix)
    X = []
    n = length(matrix[:,1])
    for i in 1:n
        append!(X, get_waiting_times(get_moments(matrix[i,:])))
    end
    
    return X
    
end

get_times_from_trajectories (generic function with 1 method)

### Generator

In [21]:
function get_waiting_times(Moments)
    
    k = length(Moments)
    Times = zeros(k-1)
    
    j = 1
    s = 0
    
    Times[1] = Moments[1]
    
    for j in 2:k-1
        
        Times[j] = Moments[j] - Moments[j-1]
        j += 1
        
    end
    
    return Times
    
end

get_waiting_times (generic function with 1 method)

### Estymator parametru $\lambda$ 

In [22]:
function estimator_λ(Times)
    
    return length(Times)/sum(Times)
    
end

estimator_λ (generic function with 1 method)

## Rozkład wypłat odszkodowań

Pomysł: Postąpimy podobnie jak w przypadku szacowania parametru intensywności procesu Poissona. Tym razem zapiszemy jednak różnicę w wartości kapitału firmy.

### Wektor $X_i$

Algorytm

1. Ustal $I = 1$, trajektorię ryzyka $R$ oraz pusty wektor $X.$
2. Zwiększ $I$ o jeden.
3. Wyznacz $\Delta R = R[I-1] - R[I]$.
3. Jeśli $\Delta R >= 0$, wstaw wynik do wektora $V$.
4. Powtarzaj 2-3, dopóki $I < \#R$.
5. Zwróć $X.$

In [23]:
function get_pays(trajectory)
    
    I = 1
    X = []
    k = length(trajectory)
    
    while I < k

        I += 1
        ΔR = trajectory[I-1] - trajectory[I] 
        if ΔR>=0
            push!(X, ΔR)
        else
            continue
        end

    end
    
    return X

end

get_pays (generic function with 1 method)

In [24]:
function get_pays_from_trajectories(matrix)
    X = []
    n = length(matrix[:,1])
    for i in 1:n
        append!(X, get_pays(matrix[i,:]))
    end
    
    return X
    
end

get_pays_from_trajectories (generic function with 1 method)

### Importuj trajektorie

In [25]:
py"""
import pickle
 
def load_pickle(fpath):
    with open(fpath, "rb") as f:
        data = pickle.load(f)
    return data
"""

load_pickle = py"load_pickle"

PyObject <function load_pickle at 0x000000005E0CC790>

In [26]:
trajectories = load_pickle("zestaw23.p")

50×10001 Matrix{Float64}:
 50.0  50.1875  50.375   50.5625  …  416.741  416.929  417.116  417.304
 50.0  50.1875  50.375   50.5625     274.982  275.169  275.357  275.544
 50.0  50.1875  50.375   50.5625     235.499  235.687  235.874  236.062
 50.0  50.1875  50.375   36.1198     245.128  245.316  245.503  245.691
 50.0  50.1875  50.375   50.5625     335.434  335.621  335.809  326.1
 50.0  39.0528  39.2403  39.4278  …  453.733  453.921  454.108  454.296
 50.0  50.1875  50.375   50.5625     526.431  526.619  526.806  526.994
 50.0  50.1875  50.375   50.5625     693.937  694.124  694.312  694.499
 50.0  50.1875  50.375   50.5625     354.827  355.014  355.202  355.389
 50.0  50.1875  50.375   50.5625     579.662  579.849  580.037  580.224
 50.0  50.1875  50.375   50.5625  …  702.636  702.823  703.011  703.198
 50.0  50.1875  33.7496  33.9371     207.181  207.368  207.556  207.743
 50.0  50.1875  50.375   50.5625     410.44   410.627  410.815  411.002
  ⋮                               ⋱     

In [27]:
X = get_pays_from_trajectories(trajectories)

7289-element Vector{Any}:
  4.004463717447408
  8.795070331404332
 18.768288397303223
  6.38123033910297
  1.2582251613792153
 32.67000636505698
 13.150728296393225
  3.2905972664473637
  1.4269458946803155
  0.5358526016019596
  3.8228812320395917
  2.739119185437616
 13.78863806881077
  ⋮
 12.582367538509743
  7.5817403000769445
 18.464297223357335
 40.36332627403476
 11.019648459679729
  5.290798799540198
  6.834277084287123
 19.15011588728362
  9.289293168463246
  5.5656469565067255
  9.160510271665999
 30.41099893881801

In [28]:
N = get_times_from_trajectories(trajectories)

7239-element Vector{Any}:
 0.46000000000000024
 0.44000000000000034
 0.2100000000000002
 0.6400000000000006
 1.1099999999999817
 0.049999999999998934
 0.5699999999999878
 0.12999999999999723
 0.20999999999999552
 0.6499999999999866
 0.33999999999999275
 0.20999999999999552
 2.3299999999999503
 ⋮
 0.3600000000001842
 0.260000000000133
 0.5300000000002711
 0.9900000000005065
 1.4500000000007418
 1.0200000000005218
 0.1800000000000921
 2.140000000001095
 0.2500000000001279
 1.210000000000619
 0.1900000000000972
 0.3700000000001893

In [29]:
minimum(N)

0.00999999999999801

In [30]:
maximum(N)

6.730000000003443

### Wizualizacja danych

In [31]:
histogram(X, normalize=true, legend=false, xlabel="wielkość wypłaty", ylabel="częstość")
savefig("histogram_wypłat.pdf")

In [32]:
histogram(N, normalize=true, xlabel="czas oczekiwania", ylabel="częstość", legend=false)
savefig("histogram_czasów.pdf")

### Oszacowanie parametru $\mu$ rozkładu $X_i$

In [33]:
μ = estimator_λ(X)

0.09901122812369685

### Oszacowanie intensywności $\lambda$ procesu $N(t)$

In [34]:
λ = estimator_λ(N)

1.4681753373312858

### Test Kołmogorowa-Smirnova

Test KS jest naturalny w użyciu. Jeżeli próbka pochodzi z jakiegoś rozkładu, to jej dystrybuanta empiryczna nie powinna się za bardzo różnić od teoretycznej. W związku z tym będziemy badać maksymalną odległość pomiędzy tymi dwoma funkcjami. Jeżeli będzie za duża - odrzucimy pomysł i zastanowimy się nad następnym. 

W związku z powyższym użyjemy statystyki

$$D_n = \max_x|F_n(x)-F(x)|,$$

gdzie:

- $F_n(x)$ - dystrybuanta empiryczna pewnej próby losowej $X$,
- $F(x)$ - teoretyczna dystrybuanta sprawdzanego rozkładu.

Można pokazać, że dla $n\rightarrow\infty$

$$\sqrt{n}D_n \rightarrow \sup_t|B(F(t))|,$$

gdzie $B$ to most Browna. Ponadto, jeśli $F$ jest ciągła, to $$\sup_t|B(F(t))|=\sup_{t\in[0,1]}|B(F(t))|$$.
Wtedy mamy do czynienia z rozkładem Kołmogorowa, niezależnym od $F$.

Pokażemy, że $D_n$ można zapisać w postaci $\max_j \{D^{+}, D^{-}\}$, gdzie:

- $D^{+} = \max_j\{\frac{j}{n} - Z_j\}$,
- $D^{-} = \max_j\{Z_j - \frac{j-1}{n}\}$.

DOWÓD.

Algorytm:

1. Ustal próbę losową $X$ o długości $n$.
2. Posortuj rosnąco $X$.
3. Wyznacz zbiór $Z = \{F(X_i), i = 1, 2, ..., n\}$ dla $X_i \in X$.
4. Oblicz $D^{+}$ oraz $D^{-}$
5. Zwróć $\max_j \{D^{+}, D^{-}\}$

In [35]:
function Dn(sample, d=Exponential(1/μ))
    n = length(sample)
    Z = cdf(d, sort(sample))
    
    D_p = zeros(n)
    D_m = zeros(n)
    
    for j in 1:n
        
        D_p[j] = abs(j/n - Z[j])
        D_m[j] = abs(Z[j] - (j-1)/n)
        
    end
    return sqrt(n)*maximum([maximum(D_p), maximum(D_m)])
    
end

Dn (generic function with 2 methods)

In [36]:
z= Dn(X)

0.9153179009882341

In [37]:
Y = (1:length(X)) ./ length(X)
xs = 0:maximum(X)/length(X):maximum(X)
ys = [cdfexponential(t, μ) for t in xs]
plot(sort(X), Y, xlabel="wielkość wypłaty", ylabel="wartość dystrybuanty", label="dystrybuanta empiryczna", legend=:bottomright)
plot!(xs, ys, label="dystrybuanta teoretyczna")
savefig("Porównanie_dystrybuant.pdf")

### Wartość krytyczna

dla $\alpha = 0.05$ jest równa $q = 1.094$

### p-wartość

Obliczymy p-wartość symulacyjnie. Niestety, nie znamy dokładnych wartości parametru pożądanego rozkładu, więc za każdym razem będziemy musieli je estymować.

Plan jest taki, aby generować $l$ prób z rozkładu wykładniczego $Exp(\mu)$ i sprawdzić, ile razy zajdzie nierówność:

$$P(D \ge z),$$

gdzie $z$ to nasza statystyka testowa z pierwotnej próby.

Algorytm:


1. Wyestymuj parametr $\mu$ rozkładu wykładniczego.
2. Wyznacz statystykę testową $z$ z pierwotnej próby.
3. Ustal $n$ (długość próby) oraz $s = 0$.
4. Generuj $X \sim Exp(\mu)$ o długości $n$.
5. Wyestymuj $\lambda$.
6. Wyznacz statystykę $D$ za pomocą $X$.
7. Sprawdź, czy zachodzi $D \ge z$. Jeśli tak, zwiększ $s$ o jeden.
8. Powtarzaj 4-7 $l$ razy.
9. Zwróć $P = \frac{s}{l}$.

In [38]:
function get_pvalue(μ, z, n=1000, l=10000)
    
    s = 0
    λ = 1/μ
    for i in 1:l
        
        X = rand(Exponential(λ), n)
        γ = estimator_λ(X)
        check = Dn(X, Exponential(1/γ))
        if check >= z
                
            s+= 1
        end
        
    end
    
    return s/l
    
end

get_pvalue (generic function with 3 methods)

In [39]:
get_pvalue(μ, z, 7000, 10000)

0.1614

### Test Andersona-Darlinga

Test AD jest bardziej odporną na odstępstwa w ogonach rozkładu wersją testu Craméra-von Misesa (odwołać się),
którego statystyka jest dana wzorem:

$$n\int_{-\infty}^{+\infty} \frac{(F_n(x) - F(x))^2}{F(x)(1-F(x))}dF(x),$$

gdzie:

- $n$ - długość próby,
- $F_n(x)$ - wartość dystrybuanty empirycznej w punkcie x,
- $F(x)$ - wartość dystrybuanty testowanego rozkładu w punkcie x.

Do obliczenia tej statystyki będziemy używać jednak innej wersji tego wzoru (odwołać się):

$$A^2 = (2i-1)\cdot log(Z_i) + (2n+1-2i)\cdot log(1-Z_i),$$

gdzie $Z$ jest posortowanym zbiór $\{F(X_i), i=1,2,..,n\}$. $X_i$ to kolejne elementy naszej próby losowej.
Ponieważ dysponujemy wyjątkowo dużą próbą, nie musimy nanosić żadnych poprawek na wartość tej statystyki.

Algorytm:

1. Wyznacz próbę losową $X$ oraz zakładany rozkład.
2. Jeśli znasz jego parametry, przejdź do kroku 3. W przeciwnym razie wyestymuj je.
3. Wyznacz posegregowany zbiór wartości dystrybuanty zakładanego rozkładu $Z$ za pomocą próby losowej $X$.
4. Oblicz $A^2$.

In [40]:
function A(Sample, d=Normal(200, 35))
    n = length(Sample)
    Z = sort(cdf(d, Sample))
  
    s = 0
    
    for i in 1:n
        s += (2*i-1)*log(Z[i]) + (2*n+1-2*i)*log(1-Z[i])
    end

    return -n - s/n
end

A (generic function with 2 methods)

### p-wartość

Obliczymy p-wartość symulacyjnie. Niestety, nie znamy dokładnych wartości parametru pożądanego rozkładu, więc za każdym razem będziemy musieli je estymować.

Plan jest taki, aby generować $l$ prób z rozkładu wykładniczego $Exp(\mu)$ i sprawdzić, ile razy zajdzie nierówność:

$$P(A^2 \ge z),$$

gdzie $z$ to nasza statystyka testowa z pierwotnej próby.

Algorytm:


1. Wyestymuj parametr $\mu$ rozkładu wykładniczego.
2. Wyznacz statystykę testową $z$ z pierwotnej próby.
3. Ustal $n$ (długość próby) oraz $s = 0$.
4. Generuj $X \sim Exp(\mu)$ o długości $n$.
5. Wyestymuj $\lambda$.
6. Wyznacz statystykę $A^2$ za pomocą $X$.
7. Sprawdź, czy zachodzi $A^2 \ge z$. Jeśli tak, zwiększ $s$ o jeden.
8. Powtarzaj 4-7 $l$ razy.
9. Zwróć $P = \frac{s}{l}$.

In [41]:
function get_pvalue(μ, z, n=1000, l=10000)
    
    s = 0
    λ = 1/μ
    for i in 1:l
        
        X = rand(Exponential(λ), n)
        γ = estimator_λ(X)
        check = A(X, Exponential(1/γ))
        if check >= z
                
            s+= 1
        end
        
    end
    
    return s/l
    
end

get_pvalue (generic function with 3 methods)

In [42]:
get_pvalue(μ, z, 7000, 10000)

0.1517

In [43]:
z = A(X, Exponential(1/μ))

1.0243430526115844

- wartość statystyki testowej $z = 1.024.$
- wartość krytyczna $\theta$ = $1.321$ dla $\alpha = 0.05$
- $p$-wartość$ = 0.109$

Wniosek: Brak podstaw do odrzucenia $H_0$.

### Wysokość premii

Premię wyznaczymy na dwa sposoby. Najpierw bardziej uniwersalnym, działającym nie tylko w przypadku klasycznego procesu ryzyka, a potem korzystając właśnie z tego faktu.

Policzmy średnią wartość straty procesu w punkcie $t$

$$E(\sum_{i=1}^{N(t)}X_i) = E(E(\sum_{i=1}^{N(t)}X_i|N(t))=E(\mu N(t)) = \mu \lambda t.$$

Gdyby przyjąć $ct = \mu \lambda t$, to otrzymalibyśmy martyngał - średnio firma utrzymywałaby stały kapitał, tzn. zarabiałaby średnio tyle, aby pokryć koszty powiązane z wypłatami odszkodowań. Ponieważ zakład ubezpieczeniowy powinien się rozwijać, to ostatecznie premię zapiszmy w postaci

$$ct = (1+\theta)\mu \lambda t$$,

gdzie $\theta$ to tzw. narzut. Im większy narzut, tym większe zarobki. Interpretujemy ten parametr jako koszt polis ubezpieczeniowych. Oczywiście zakładamy, że $\theta > 0$. Prawdopodobnie narzut nie jest zbyt wysoki - nikt by wtedy tak drogiej polisy nie wykupił.

Docelowo chcemy wyłuskać parametr $\theta$. Przekształcając powyższe równanie otrzymujemy

$$ \theta = \frac{c}{\mu \lambda} - 1.$$

Aby to zrobić, wyznaczymy symulacyjnie średni wzrost kapitału firmy ubezpieczeniowej w czasie. Będzie on postaci $\theta \mu \lambda t$.

Ponieważ znamy już $\mu$ oraz $\lambda$, to z otrzymanego wyniku łatwo będzie uzyskać $\theta$.

In [44]:
function get_mean(trajectories::Matrix)
    
    n = length(trajectories[1, :])
    m = length(trajectories[:, 1])
    Means = zeros(n)
    
    for index in 1:m
        Means += trajectories[index, :]
    end
    
    return Means ./ m
    
end
    

get_mean (generic function with 1 method)

In [45]:
mean_trajectory = get_mean(trajectories)

10001-element Vector{Float64}:
  50.0
  49.827365941496545
  49.682357497584235
  49.58100313021396
  49.49785615971196
  49.6796067263617
  49.8671067263617
  50.031360814520525
  50.218860814520525
  50.31952096299896
  49.42970409846479
  49.61720409846479
  48.60290792909229
   ⋮
 424.13360131093873
 424.09935105104114
 424.27614432576746
 424.46364432576746
 424.6511443257678
 424.7277495134533
 424.9152495134533
 424.89573002672097
 424.719829620578
 424.907329620578
 425.09482962057837
 425.08440965608287

In [46]:
xs = 0:h:T

0.0:0.01:100.0

#### Regresja liniowa

In [47]:
np = pyimport("numpy")

PyObject <module 'numpy' from 'C:\\Users\\EpicPC\\.julia\\conda\\3\\lib\\site-packages\\numpy\\__init__.py'>

In [48]:
polyfit = np.polyfit

PyObject <function polyfit at 0x00000000649E5670>

In [49]:
vec = polyfit(xs, mean_trajectory, 1)

2-element Vector{Float64}:
  3.746001609231775
 29.686954594543682

In [50]:
f(x) = vec[1]*x

f (generic function with 1 method)

In [51]:
ys_mean = [f(t)+50 for t in [0,100]]

2-element Vector{Float64}:
  50.0
 424.6001609231775

In [52]:
plot(xs, mean_trajectory, ylim=[0,500], label="empiryczna średnia procesu", color=:red, legend=:topleft)
plot!([0, 100], ys_mean, label="teoretyczna średnia procesu", xlabel="czas t", ylabel="wartość procesu", color=:green)
savefig("")

Więc $$\theta \mu \lambda = 3.746$$

Po podstawowych przekształceniach i podstawieniu wartości $\mu$ oraz $\lambda$ pokazujemy, że

$$\theta = 25.769$$

Ostatecznie $$R(t) = 50 + 3.891t - \sum_{i=1}^{N(t)}X_i$$

In [53]:
μ

0.09901122812369685

In [54]:
λ

1.4681753373312858

In [55]:
θ= vec[1]*μ/(λ)

0.2526239274374162

### 100 trajektorii

In [56]:
xs = 0:h:T-h
xs1 = 0:h:T
ys = risk_process(50, λ=λ, μ=μ, θ=θ)

plot(xs, ys, color=:blue, label="wiązka trajektorii", xlabel="czas t", ylabel="wartość procesu")

for i in 1:99
    plot!(xs, risk_process(50, λ=λ, μ=μ, θ=θ), color=:blue, label="")
end
plot!(xs1, mean_trajectory, color=:yellow, label="średnia procesu z zestawu danych", linewidth=3)
plt = plot!(xs1, trajectories[1,:],color=:red, label="Trajektoria z zestawu danych", legend=:topleft, linewidth=3)
savefig("100_trajektorii_ryzyka.pdf")

### Porównanie średnich wartości procesu

In [57]:
function get_matrix(N::Int, u::Real; θ::Float64=0.5, μ::Float64=1.0, λ::Float64=1.0, T::Int=100, h::Float64=0.01)
    
    k::Int = round(T/h)
    Trajectories = zeros(N, k)
    
    for i in 1:N
        Trajectories[i, :] = risk_process(u, θ=θ, μ=μ, λ=λ, T=T, h=h)
    end
    
    return Trajectories
    
end

get_matrix (generic function with 1 method)

In [58]:
emp_trajectories = get_matrix(50, 50,θ=θ, μ=μ, λ=λ, T=T, h=h)
emp_mean = get_mean(emp_trajectories)

plot(xs, emp_mean, legend=:topleft,ylim=[0,450], label="średnia empiryczna", color=:blue, xlabel="czas t", ylabel="wartość procesu")
plot!(xs1, mean_trajectory, label="średnia z zestawu danych", color=:red)
plot!([0,100], ys_mean, label="średnia procesu",color=:green)
savefig("Porównanie_średnich_procesu_ryzyka.pdf")

### Prawdopodobieństwo bankructwa w skończonym czasie T

In [59]:
function finite(N::Int, u::Real; θ::Float64=0.5, μ::Float64=1.0, λ::Float64=1.0, T::Int=100, h::Float64=0.01)
    
    s = 0
    
    for i in 1:N
        if any(x -> x<=0, risk_process(u, θ=θ, μ=μ, λ=λ, T=T, h=h))
            s += 1
        end
    end
    
    return s/N
        
end

finite (generic function with 1 method)

In [60]:
finite(10000, 50; θ=θ, μ=μ, λ=λ, T=100, h=h)

0.2945

In [61]:
finite(10000, 50; θ=θ, μ=μ, λ=λ, T=200, h=h)

LoadError: BoundsError: attempt to access 20000-element Vector{Float64} at index [20001]

$T = 100$ to $P = 0.2917$


$T = 200$ to $P = 0.2886$


$T \rightarrow +\infty$ to $P = 0.29415$

### Prawdopodobieństwo bankructwa w nieskończonym czasie

- Wzór Pollaczka-Chinczyna

Algorytm:

1. Generuj $K \sim Geom(\frac{\theta}{1+\theta})$
2. Generuj $Y_1, Y_2, ..., Y_k$ - $i.i.d$ o gęstości $f(x) = \frac{1-F_X(x)}{EX}$
3. Jeśli $\sum_{i=1}^{k}Y_i > \mu$, wstaw $Z(i) = 1$. W przeciwnym wypadku $Z(i) = 0$.
4. Powtarzaj kroki 1-3 $N$ razy.
5. Zwróć $\Psi(u) = \frac{\sum_{i=1}^{N}Z_i}{N}$


#### Generowanie zmiennej losowej z rozkładu geometrycznego

Algorytm:
1. Generuj $U \sim U(0, 1)$.
2. Zwróć $X = \lceil \frac{ \log{U} } { \log{1-p} } \rceil$

In [62]:
function geom(p)
    U = rand()
    
    return ceil(log(U)/log(1-p))
end

geom (generic function with 1 method)

In [63]:
function infinite(u::Int, μ::Float64, θ::Float64; N=10000)
    
    p = θ/(1+θ)
    s = 0
    
    for i in 1:N
        K::Int = rand(Geometric(p))
        Ys = rexponential(μ, K)
        if sum(Ys) > u
            s += 1
        end
    end
    
    return s/N
    
end

infinite (generic function with 1 method)

In [64]:
infinite(50, μ, θ, N=100000000)

0.29417051

In [65]:
function infinite_theoretical(u, μ, θ)
    
    return ℯ^(-μ*u*θ/(1+θ))/(1+θ)
    
end

infinite_theoretical (generic function with 1 method)

In [66]:
infinite_theoretical(u, μ, θ)

0.294154863265306